In [1]:
import datetime
import sys
import pymc3 as pm
import theano.tensor as tt
import pickle
import pandas as pd
import numpy as np
import scipy.special
from multiprocessing import cpu_count


sys.path.append("../covid19_inference")
sys.path.append("..")

import data
from causal_covid.data import load_cases, load_vaccinations, infectiability
from causal_covid.utils import get_cps, day_to_week_matrix
from covid19_inference import Cov19Model
from covid19_inference.model import (
    lambda_t_with_sigmoids,
    kernelized_spread_gender,
    week_modulation,
    student_t_likelihood,
    delay_cases,
    uncorrelated_prior_E,
)

begin = datetime.datetime(2021,8,1)
end = datetime.datetime(2021,10,1)
file = "./../data/2022-02-09_16-39-19_young_to_old_cap/scenario_export/observed_infection_data.csv"
cases_df = load_cases(file, begin, end)


vaccination_file = "./../data/2022-02-09_16-39-19_young_to_old_cap/scenario_export/observed_vaccination_data.csv"
population_file = "./../data/2022-02-09_16-39-19_young_to_old_cap/scenario_export/population_data.csv"
observed_U2_file = "./../data/2022-02-02_14-19-24_observed_vac_policy/vaccination_policy/U_2.npy"
observed_U3_file = "./../data/2022-02-02_14-19-24_observed_vac_policy/vaccination_policy/u_3.npy"
scenario_U2_file = "./../data/2022-02-09_16-39-19_young_to_old_cap/vaccination_policy/U_2.npy"
scenario_U3_file = "./../data/2022-02-09_16-39-19_young_to_old_cap/vaccination_policy/u_3.npy"

vaccination_dfs = load_vaccinations(vaccination_file, population_file, begin, end)

infectiability = infectiability(vaccination_dfs, [.7,.7,.7], vaccination_dfs[1:])

In [2]:
cases_df

,0-29,30-59,60+
2021-08-01,12118.0,8052.0,3326.0
2021-08-08,19348.0,13693.0,4974.0
2021-08-15,28228.0,18120.0,4721.0
2021-08-22,36416.0,20764.0,3755.0
2021-08-29,43083.0,19677.0,2982.0
2021-09-05,37803.0,15010.0,2145.0
2021-09-12,37234.0,14876.0,2185.0
2021-09-19,27093.0,11286.0,1866.0
2021-09-26,16557.0,7082.0,1245.0


In [3]:
infectiability

,0-29,30-59,60+
2021-08-01,0.923509,0.546033,0.680763
2021-08-08,0.922171,0.561869,0.710972
2021-08-15,0.920105,0.657248,0.661734
2021-08-22,0.917915,0.743638,0.626199
2021-08-29,0.936695,0.765403,0.603643
2021-09-05,0.941489,0.765629,0.594632
2021-09-12,0.944364,0.762028,0.584168
2021-09-19,0.945063,0.756847,0.574728
2021-09-26,0.947231,0.742457,0.560344


In [4]:
vaccination_dfs

(                0-29     30-59       60+
 2021-08-01  0.630100  0.142670  0.077788
 2021-08-08  0.624365  0.140840  0.076273
 2021-08-15  0.616503  0.137836  0.074334
 2021-08-22  0.606759  0.133786  0.072506
 2021-08-29  0.597408  0.129423  0.070972
 2021-09-05  0.593576  0.127500  0.070197
 2021-09-12  0.589897  0.125386  0.069414
 2021-09-19  0.586344  0.123186  0.068653
 2021-09-26  0.581251  0.118445  0.067321,
                 0-29     30-59       60+
 2021-08-01  0.042017  0.053820  0.031976
 2021-08-08  0.045281  0.054522  0.031513
 2021-08-15  0.049269  0.055208  0.031494
 2021-08-22  0.055296  0.056774  0.031757
 2021-08-29  0.058816  0.058300  0.031594
 2021-09-05  0.057506  0.058445  0.031258
 2021-09-12  0.055201  0.058275  0.030441
 2021-09-19  0.052989  0.057997  0.029625
 2021-09-26  0.053526  0.059463  0.029041,
                 0-29     30-59       60+
 2021-08-01  0.327883  0.803510  0.616539
 2021-08-08  0.330354  0.789106  0.362797
 2021-08-15  0.334229  0.687211 